In [1]:
import re

import pandas as pd
import plotly.express as px

from Project.Database import Db

In [ ]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)
year2_minutes = Db.load_data(hourly=False, year=2)

In [3]:
consumers = meta.loc[meta["Units"] == "W"].index.tolist()
status_cols = meta.loc[(lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self.index.str.contains("Child")) &
        (~self.index.str.contains("Prnt"))
))].index.tolist()

In [4]:
year1_minutes = year1_minutes.loc[:4319, ["Timestamp"] + consumers + status_cols].copy()
year1_minutes

,Timestamp,DHW_HeatPumpWaterHeaterPowerTotal,DHW_SHWPumpsPowerWithStandby,Elec_PowerChildADownstairs,Elec_PowerChildAUpstairs,Elec_PowerChildBDownstairs,Elec_PowerChildBUpstairs,Elec_PowerClothesWasher,Elec_PowerDishwasher,Elec_PowerDryer1of2,...,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadVacuum,Load_StatusPlugLoadVideoGame,Load_StatusRefrigerator,Load_StatusMicrowave,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusHeatPumpWaterHeater
0,2013-07-01 00:01:14,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0
1,2013-07-01 00:02:14,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0
2,2013-07-01 00:03:14,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0
3,2013-07-01 00:04:14,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0
4,2013-07-01 00:05:14,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2013-07-04 00:23:25,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
4316,2013-07-04 00:24:25,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
4317,2013-07-04 00:25:25,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
4318,2013-07-04 00:26:25,6.92,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0


In [5]:
appliance_dict = {}
for appliance in status_cols:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load")[-1]))
    appliance_switch = year1_minutes[(year1_minutes[appliance] != year1_minutes[appliance].shift(1))][
                           ["Timestamp", appliance]][1:]
    appliance_dict[name] = pd.DataFrame(
        {"Timestamp": [appliance_switch.loc[index, "Timestamp"] for index in appliance_switch.index],
         "Appliance": name,
         "Color": ["green" if appliance_switch.loc[index, appliance] == 1 else "red" for index in
                   appliance_switch.index]})

appliance_dict

{'Status Appliance Cooktop':             Timestamp                 Appliance  Color
 0 2013-07-01 18:00:31  Status Appliance Cooktop  green
 1 2013-07-01 18:17:31  Status Appliance Cooktop    red
 2 2013-07-02 18:00:28  Status Appliance Cooktop  green
 3 2013-07-02 18:17:28  Status Appliance Cooktop    red
 4 2013-07-03 18:00:27  Status Appliance Cooktop  green
 5 2013-07-03 18:17:27  Status Appliance Cooktop    red,
 'Status Appliance Dishwasher':             Timestamp                    Appliance  Color
 0 2013-07-01 20:26:31  Status Appliance Dishwasher  green
 1 2013-07-01 22:36:30  Status Appliance Dishwasher    red
 2 2013-07-03 20:26:27  Status Appliance Dishwasher  green
 3 2013-07-03 22:37:27  Status Appliance Dishwasher    red,
 'Status Appliance Oven': Empty DataFrame
 Columns: [Timestamp, Appliance, Color]
 Index: [],
 'Status Appliance Range Hood':             Timestamp                    Appliance  Color
 0 2013-07-01 18:00:31  Status Appliance Range Hood  green
 1 2013-0

In [ ]:
for i in consumers:
    fig = px.scatter(year1_minutes, x=year1_minutes["Timestamp"], y=i)

    height = 1
    for appliance, df in appliance_dict.items():
        for index in df.index:
            x = df.loc[index, "Timestamp"].timestamp() * 1000 - (2 + int(str(df.loc[index, "Timestamp"]).split("-")[-1][
                                                                             1])) * 3_600_000  #Plotly bug work around. Computes unix time with offset
            fig.add_vline(x=x,
                          line_width=3,
                          line_dash="dash",
                          annotation_text=df.loc[index, "Appliance"],
                          line_color=df.loc[index, "Color"],
                          annotation_y=height)
        height -= 0.05

    fig.update_layout(
        xaxis={'type': 'date', 'tickformat': '%Y-%m-%d %H:%M:%S'}
    )

    fig.update_layout(
        xaxis_tickformatstops=[
            dict(dtickrange=[None, 60000], value="%H:%M:%S\n%d %b %y"),
            dict(dtickrange=[60000, 3600000], value="%H:%M\n%d %b %y"),
            dict(dtickrange=[3600000, 86400000], value="%d %b %Y")
        ]
    )
# fig.write_html(Db.get_save_file_directory("Kitchen_plug_appliances_consumption.html"))
    fig.show()

KeyboardInterrupt: 